<a href="https://colab.research.google.com/github/EvgeniyBogatyrev/makingThings/blob/main/ParserGA_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import re
import requests
from bs4 import BeautifulSoup
import csv
import time

In [ ]:
def get_brand_data(url):
  with open(f"A_Z_brands_products_price.csv", "w", encoding="utf-8-sig") as file:
                        writer = csv.writer(file)
                        writer.writerow(
                            (
                              "Бренд продукта",
                              "Наименование продукта",
                              "Категория",
                              "В продаже",
                              "Цена продукта",
                              "Цена при макс скидке",
                              "Объем"
                            )
                        )
  with open(f"net_URL.csv", "w", encoding="utf-8-sig") as file:
                        writer = csv.writer(file)
                        writer.writerow(
                            (
                              "Бренд продукта",
                              "Ссылка"
                            )
                        )                    
  
  headers = {
      "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"
  }

#Запрос к сайту, выкачали страничку и сохранили
  req = requests.get(url, headers)
  src = req.text
    

  soup = BeautifulSoup(src, "lxml")
  p = re.search(r'"brands":\ (.*?)]   ', src).group(1)+"]"
  brandDictionary = json.loads((p))
  
  for x in brandDictionary:

      for i in x:
          print(f"Бренды под буквой {i}:")
      
          for j in x[i]:
              label = j.get("label")
              url = j.get("url")
              req = requests.get(url, headers)
              src = req.text

              soup = BeautifulSoup(src, "lxml")

              product_container = soup.find_all(text=re.compile("(productsApiUrl)"))
              myString = ''.join(product_container)
              
              match = re.search("(?P<url>https?://[^\s'\"]+)", myString)
              
              if match:
                match_found = match.group("url")

                count = 0
                for k in range(1,10):
                  page = f"&page={k}"
                  req1 = requests.get(f"{match_found}+{page}", headers)
                  src1 = req1.text
                  productDictionary = json.loads((src1))

                  empty = []
                  empty2 = {'title': 'Just For You'}
                  
                  if productDictionary == empty or productDictionary == empty2:
                    print(f"Бренд {label}. Скопированы данные для {count} продуктов.")
                    break

                  for i in productDictionary["products"]:

                        product_brand = i.get('brand')
                        product_name = i.get('name')
                        product_category = i.get('category_type')
                        product_is_seleable = i.get('is_saleable')
                        price = i['price_object']['amount']
                        best_price = i.get('best_loyalty_price')
                        volume = i.get('volume')

                        with open(f"A_Z_brands_products_price.csv", "a", encoding="utf-8-sig") as file:
                            writer = csv.writer(file)
                            writer.writerow(
                                (
                                  product_brand,
                                  product_name,
                                  product_category,
                                  product_is_seleable,
                                  price,
                                  best_price,
                                  volume
                                )
                            )
                        count +=1
                time.sleep(1)
              else:
                print("url не найден")
                with open(f"net_URL.csv", "a", encoding="utf-8-sig") as file:
                            writer = csv.writer(file)
                            writer.writerow(
                                (
                                  label,
                                  url
                                )
                            )
                with open(f"{label}.html", "w") as file:
                  file.write(req.text)
                with open(f"{label}.html") as file:
                  src = file.read() 

get_brand_data("https://goldapple.ru/brands")